In [3]:
 ! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [4]:
import spacy
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import pipeline

/opt/anaconda3/envs/ai_diary/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# pip install sentence-transformers

In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

# Load the model
model_name = "AI4Bharat/IndicNER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create an NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Test with an Indian diary entry
text = "Yesterday, I visited Mumbai and met with Ramesh at the Taj Hotel."

# Get entities
entities = ner_pipeline(text)
print(entities)

/opt/anaconda3/envs/ai_diary/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


[{'entity': 'B-LOC', 'score': 0.9686415, 'index': 5, 'word': 'mumbai', 'start': 21, 'end': 27}, {'entity': 'B-PER', 'score': 0.9260834, 'index': 9, 'word': 'rame', 'start': 41, 'end': 45}, {'entity': 'B-LOC', 'score': 0.96747255, 'index': 13, 'word': 'taj', 'start': 55, 'end': 58}, {'entity': 'I-LOC', 'score': 0.94324523, 'index': 14, 'word': 'hotel', 'start': 59, 'end': 64}]
